# ![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=100&v=4>)Legacy MIPS map homogenisation


The original MIPS maps have been produced by IPAC. This code takes the individual image files and adds them into a single multi-layered fits files to have the homogeneous HELP format.  

## Caveats 
* I've run this on the MIPS 24/70/160 micron maps
* only puts in image, error and coverage files (and adds blank PSF extension)

<h2> Guide</h2>
<h3>Steps: </h3>
* Obtain list of fields from sub-folders
for each field:
* read in coverage, image and noise
* attach meta data
* create PSF extension
* Save out

<h3>Things you may wish to edit</h3>
* You'll need to change the folder path in the 2nd code block to point you where the files are in your system
* Add in a specifc field (or switch to wild chracter) in the path to pick field(s)
* read in more of the available files and add as additional extensions?????


# Importing libraries

In [11]:
from datetime import datetime
from itertools import product
import glob

import numpy as np
import astropy as astropy

from astropy.io import fits
from astropy.table import Table
from astropy.table import Column
from astropy import wcs

import shutil

VERSION = "0.9"

# This is where you set up the field and folders to work on
the notebook needs to be run for each field in turn - commenting in/out as appropriate

In [4]:
#field_specified='HDF-N' 
#field_specified='XMM-LSS' 
#field_specified='AKARI-NEP' 
#field_specified='AKARI-SEP' 
#field_specified='Bootes' 
#field_specified='CDFS-SWIRE' 
#field_specified='COSMOS' 
#field_specified='EGS' 
#field_specified='ELAIS-N1' 
field_specified='ELAIS-N2' 
#field_specified='ELAIS-S1' 
#field_specified='GAMA-09' 
#field_specified='GAMA-12' 
#field_specified='GAMA-15' 
#field_specified='NGP' 
#field_specified='SGP' 
#field_specified='Lockman-SWIRE' 
#field_specified='SA13' 
#field_specified='SPIRE-NEP' 
#field_specified='SSDF' 
#field_specified='xFLS' 
#field_specified='XMM-13hr' 
#field_specified='Herschel-Stripe-82' 


mainfolder='../../dmu17/dmu17_Legacy_maps/'
moc_dir=   '../../dmu2/dmu2_field_coverages/'

# finding all the Legacy mosaic files in the folder

In [5]:
# finding all the Legacy mosaic files (other files should be there if the mosaic file is)
filelist= glob.glob(f"{mainfolder}/{field_specified}/data/*mosaic.fits", recursive=True)
print("Number of files found:", len(filelist))

Number of files found: 3


# Performing the homogenisation
i.e. merging component files into a single multi-extension file and adding some keywords
traps errors so that it can continue, e.g. if a file is missing
skips over files if they've already been logged as "downloaded" (this could be improved to be an option

In [6]:
for i, file in enumerate(filelist):
    print('----------------')
    print(i,file)
    folders=file.split('/')
    print(folders[len(folders)-3])
    field=folders[len(folders)-3]
    filename=folders[len(folders)-1]
    ext=0
    
    # Load in image map
    image_map=fits.open(file)
    image_hdu = fits.ImageHDU(header=image_map[ext].header,
                                  data=image_map[ext].data)
    image_hdu.header['EXTNAME'] = "IMAGE"
 
        # Load in uncertainty map
    noise_map=fits.open(file.replace('mosaic','mosaic_unc'))
    noise_hdu = fits.ImageHDU(header=noise_map[ext].header,
                                data=noise_map[ext].data)
    noise_hdu.header['EXTNAME'] = "ERROR"
 
     
            # Load in coverage map
    cov_map=fits.open(file.replace('mosaic','mosaic_cov'))
    cov_hdu = fits.ImageHDU(header=cov_map[ext].header,
                                data=cov_map[ext].data)
    cov_hdu.header['EXTNAME'] = "COVERAGE"

            
            #Create primary header
        
    CHNLNUM=image_hdu.header['CHNLNUM'] 
    primary_hdu = fits.PrimaryHDU()
    primary_hdu.header.append((
            "CREATOR", "Herschel Extragalactic Legacy Project"
        ))
    primary_hdu.header.append((
            "TIMESYS", "UTC", "All dates are in UTC time"
        ))
    primary_hdu.header.append((
            "DATE", datetime.now().replace(microsecond=0).isoformat(),
            "Date of file creation"
        ))
    primary_hdu.header.append((
        "VERSION", VERSION, "HELP product version"
        ))
    primary_hdu.header.append((
        "TELESCOP", "Spitzer", "Name of the telescope"
        ))
    primary_hdu.header.append((
        "INSTRUME", "MIPS", "Name of the instrument"
        ))
    primary_hdu.header.append((
        "FILTER", CHNLNUM, "1=24um,2=70um,3=160um"
        ))
    primary_hdu.header.append((
        "FIELD", field, "Name of the HELP field"
        ))


        #Add empty extension for PSF
    psf_hdu = fits.ImageHDU()
    psf_hdu.header['EXTNAME'] = "PSF"
    psf_hdu.header.add_comment("The PSF is not available.")

    hdu_list = fits.HDUList([primary_hdu, image_hdu, noise_hdu, cov_hdu, psf_hdu])
    hdu_list.writeto(file.replace('mosaic','help'),
                 checksum=True, overwrite=True)
    print(f"{file} /  processed...")

----------------
0 ../../dmu17/dmu17_Legacy_maps//ELAIS-N2/data/swire_EN2_M3_v4_mosaic.fits
ELAIS-N2
../../dmu17/dmu17_Legacy_maps//ELAIS-N2/data/swire_EN2_M3_v4_mosaic.fits /  processed...
----------------
1 ../../dmu17/dmu17_Legacy_maps//ELAIS-N2/data/swire_EN2_M1_v4_mosaic.fits
ELAIS-N2
../../dmu17/dmu17_Legacy_maps//ELAIS-N2/data/swire_EN2_M1_v4_mosaic.fits /  processed...
----------------
2 ../../dmu17/dmu17_Legacy_maps//ELAIS-N2/data/swire_EN2_M2_v4_mosaic.fits
ELAIS-N2
../../dmu17/dmu17_Legacy_maps//ELAIS-N2/data/swire_EN2_M2_v4_mosaic.fits /  processed...


# Moving the files
Once we run this code, we obtain the Help Legacy Maps, in the folder: /dmu17_Legacy_maps/'. We need to put them in /dmu17_HELP_Legacy_Maps/

In [18]:
dest = f"./{field_specified}/data/"
files = glob.glob(f"{mainfolder}/{field_specified}/data/*_help.fits", recursive=True)

for f in files:
        shutil.move(f, dest)



-------------------------------------------------------------------------------

![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=75&v=4)

**Authors**:  This code was adapted from code used for SPIRE homogenisaton (Yannick Roehlly) and in turn adapted for PACS data (Peter Hurley). [Seb Oliver](http://www.sussex.ac.uk/profiles/91548)

 
For a full description of the database and how it is organised in to `dmu_products` please the top level [readme](../readme.md).
 
The Herschel Extragalactic Legacy Project, ([HELP](http://herschel.sussex.ac.uk/)), is a [European Commission Research Executive Agency](https://ec.europa.eu/info/departments/research-executive-agency_en)
funded project under the SP1-Cooperation, Collaborative project, Small or medium-scale focused research project, FP7-SPACE-2013-1 scheme, Grant Agreement
Number 607254.

[Acknowledgements](http://herschel.sussex.ac.uk/acknowledgements)

